In [1]:
import pandas as pd
numericos = ['AVProductsInstalled',
'AVProductsEnabled',
'Census_ProcessorCoreCount',
'Census_PrimaryDiskTotalCapacity',
'Census_SystemVolumeTotalCapacity',
'Census_TotalPhysicalRAM',
'Census_InternalPrimaryDiagonalDisplaySizeInInches',
'Census_InternalPrimaryDisplayResolutionHorizontal',
'Census_InternalPrimaryDisplayResolutionVertical',
'Census_InternalBatteryNumberOfCharges']
dtype = {}
for df in pd.read_csv('encoding/final_sembat.csv',low_memory=False,chunksize=10):
    for var in df.columns:
        if var not in numericos:
            dtype[var] = 'int8'
    break

In [2]:
from sklearn.model_selection import ParameterSampler
import copy
import gc
from sklearn.metrics import roc_curve, auc
def hyper_parameter_optimization(params, modeloG, testX, testY, trainX, trainY):
    lista_parametros = list(ParameterSampler(params, n_iter=10, random_state=10))
    for elem in lista_parametros:
        altera = False
        for key, value in elem.items():
            if key == 'solver':
                if value in ['newton-cg','lbfgs','sag']:
                    altera = True
        if altera == True:
            elem['penalty'] = 'l2'
    tabelaFinal = []
    for param in lista_parametros:
        print(param)
        modelo = copy.copy(modeloG)
        modelo.set_params(**param)
        #print(modelo.get_params())
        modelo.fit(trainX,trainY)
        pred = modelo.predict(testX)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(testY, pred)
        score = auc(false_positive_rate, true_positive_rate)
        elem = param.copy()
        elem['score'] = score
        tabelaFinal.append(elem)
        del modelo
        gc.collect()
    return tabelaFinal

In [3]:
import pandas as pd
auxPred = pd.DataFrame()
i = 0
for tp in pd.read_csv('encoding/final_sembat.csv',low_memory=False,chunksize=50000,dtype=dtype):
#     if i == 55:
#         break
    if i == 0:
        auxPred = pd.concat([auxPred,tp])
    else:
        auxPred = pd.concat([auxPred,tp],ignore_index=True)
    i+=1
    print(i)
#aux = pd.read_csv('encoding/predict.csv',low_memory=False,dtype=dtype)

1
2
3
4
5
6
7
8
9
10
11
12


In [4]:
trainX = auxPred.loc[:499999, auxPred.columns != 'HasDetections']
valX = auxPred.loc[500000:549999, auxPred.columns != 'HasDetections']
trainY = auxPred.loc[:499999,'HasDetections']
valY = auxPred.loc[500000:549999,'HasDetections']
print(trainX.shape)
print(valX.shape)
print(trainY.shape)
print(valY.shape)
# parametros = {'C':[0.001,0.01,0.1,1],'penalty':['l1','l2'],'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
# m = LogisticRegression(max_iter=10000)
# tabela = hyper_parameter_optimization(parametros,m,valX,valY,trainX,trainY)


(500000, 707)
(50000, 707)
(500000,)
(50000,)


In [5]:
import gc
import math
from sklearn.feature_selection import VarianceThreshold
def realizaVarThreshold():
    indices = []
    col = trainX.columns
    total = len(col)
    chunk = math.floor(total / 10)
    print(chunk)
    quantos = 0
    for i in range(chunk):
        sel = VarianceThreshold(threshold=0.001)
        try:
            sel.fit(trainX[col[quantos:quantos+10]])
            aux = [i+quantos for i in sel.get_support(indices=True)]
            indices.extend(aux)
        except:
            pass
        del sel
        gc.collect()
        quantos = quantos+10
    sel = VarianceThreshold(threshold=0.001)
    try:
        sel.fit(trainX[col[quantos:quantos+7]])
        indices.extend(sel.get_support(indices=True))
    except:
        pass
    del sel
    gc.collect()
    return indices

In [6]:
indices = realizaVarThreshold()
print(len(indices))
col = []
coln = trainX.columns
for i in indices:
    col.append(coln[i])

70
300


In [7]:
del trainX
del valX
gc.collect()

0

In [8]:
raise

RuntimeError: No active exception to reraise

In [9]:
trainX = auxPred.loc[:499999, col]
valX = auxPred.loc[500000:549999, col]

In [13]:
from sklearn.ensemble import RandomForestClassifier
n_estimators = [8, 16, 32, 64, 100]
max_features = ['auto','log2']
max_depth = [32,64,128,None]
criterion = ['gini','entropy']
parametros = {'n_estimators':n_estimators,'max_features':max_features,'max_depth':max_depth,'criterion':criterion}
m = RandomForestClassifier()
tabela = hyper_parameter_optimization(parametros,m,valX,valY,trainX,trainY)

{'n_estimators': 16, 'max_features': 'auto', 'max_depth': 32, 'criterion': 'entropy'}
{'n_estimators': 16, 'max_features': 'log2', 'max_depth': 32, 'criterion': 'entropy'}
{'n_estimators': 16, 'max_features': 'log2', 'max_depth': 128, 'criterion': 'gini'}
{'n_estimators': 32, 'max_features': 'auto', 'max_depth': None, 'criterion': 'gini'}
{'n_estimators': 8, 'max_features': 'log2', 'max_depth': None, 'criterion': 'gini'}
{'n_estimators': 8, 'max_features': 'log2', 'max_depth': 128, 'criterion': 'entropy'}
{'n_estimators': 16, 'max_features': 'auto', 'max_depth': 128, 'criterion': 'gini'}
{'n_estimators': 64, 'max_features': 'auto', 'max_depth': 32, 'criterion': 'gini'}
{'n_estimators': 32, 'max_features': 'log2', 'max_depth': None, 'criterion': 'gini'}
{'n_estimators': 100, 'max_features': 'log2', 'max_depth': 64, 'criterion': 'gini'}


In [14]:
import pandas as pd
tab = pd.DataFrame(tabela)
tab.sort_values(by='score',ascending=False)

,criterion,max_depth,max_features,n_estimators,score
7,gini,32.0,auto,64,0.667463
9,gini,64.0,log2,100,0.658444
0,entropy,32.0,auto,16,0.657307
3,gini,NaN,auto,32,0.655033
1,entropy,32.0,log2,16,0.652853
8,gini,NaN,log2,32,0.649633
6,gini,128.0,auto,16,0.643742
2,gini,128.0,log2,16,0.640022
5,entropy,128.0,log2,8,0.629723
4,gini,NaN,log2,8,0.626342


In [ ]:
raise

In [ ]:
auxPred.memory_usage().sum() / (1000*1000)

In [ ]:
auxPred.loc[0:35000, auxPred.columns != 'HasDetections'].memory_usage().sum() / (1000*1000)

In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(auxPred.loc[0:35000, auxPred.columns != 'HasDetections'],auxPred.loc[0:35000,'HasDetections']) 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
n_estimators = [1, 2, 4, 8, 16, 32, 64]#, 100, 200]
parametros = {'n_estimators':n_estimators}
m = RandomForestClassifier()
tabela = hyper_parameter_optimization(parametros,m,valX,valY,trainX,trainY)

In [ ]:
import pandas as pd
tab = pd.DataFrame(tabela)
tab.sort_values(by='score',ascending=False)

In [ ]:
import numpy as np
min_samples_splits = np.linspace(0.1, 0.3, 5, endpoint=True)
min_samples_splits

In [ ]:
min_samples_leafs = np.linspace(0, 0.2, 5, endpoint=True)
min_samples_leafs[0] = 0.00001
list(min_samples_leafs)

In [ ]:
gc.collect()

In [ ]:
tabela